In [ ]:
import os  
import xml.etree.ElementTree as ET  
import pandas as pd  
import numpy as np  
  
# Define the directory containing images and XML files  
directory = r'D:\\Github\\Projet Greta ia\\mini projet\\Moustafa\\model_fruit\\Data'  
  
# Initialize a list to hold the data  
data = []  
  
# Function to parse XML files and create a DataFrame  
def create_dataframe(directory):  
    for filename in os.listdir(directory):  
        if filename.endswith('.xml'):  
            # Parse the XML file  
            tree = ET.parse(os.path.join(directory, filename))  
            root = tree.getroot()  
              
            # Extract path to the image file  
            image_path = os.path.join(directory, root.find('filename').text)  
            # Replace backslashes with forward slashes  
            image_path = image_path.replace('\\', '/')  
              
            # Iterate over each object in the XML file  
            for obj in root.findall('object'):  
                class_id = obj.find('name').text  
                bbox = obj.find('bndbox')  
                xmin = int(bbox.find('xmin').text)  
                ymin = int(bbox.find('ymin').text)  
                xmax = int(bbox.find('xmax').text)  
                ymax = int(bbox.find('ymax').text)  
                  
                # Append the data to the list  
                data.append([image_path, xmin, xmax, ymin, ymax, class_id])  
      
    # Create a DataFrame from the data  
    df = pd.DataFrame(data, columns=['path_to_frame', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'])  
    return df  
  
# Function to map class names to numerical values  
def map_class_ids(df):  
    class_mapping = {  
        'apple': 0,  
        'banane': 1,  
        'orange': 2  
    }  
    df['class_id'] = df['class_id'].map(class_mapping)  
    return df  
  
# Create the DataFrame  
df = create_dataframe(directory)  
  
# Map class names to numerical values  
df = map_class_ids(df)  
  
# Display the DataFrame  
print(df)  


In [ ]:
import os  
import xml.etree.ElementTree as ET  
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import matplotlib.patches as patches  
from PIL import Image  
  
# Function to plot images with bounding boxes and class labels  
def plot_images_with_bboxes(df):  
    # Reverse the class mapping to get class names from class IDs  
    class_mapping = {0: 'apple', 1: 'banane', 2: 'orange'}  
      
    for idx, row in df.iterrows():  
        # Open the image  
        image = Image.open(row['path_to_frame'])  
          
        # Create a figure and axis  
        fig, ax = plt.subplots(1)  
        ax.imshow(image)  
          
        # Create a rectangle patch  
        bbox = patches.Rectangle((row['xmin'], row['ymin']),  
                                 row['xmax'] - row['xmin'],  
                                 row['ymax'] - row['ymin'],  
                                 linewidth=2,  
                                 edgecolor='r',  
                                 facecolor='none')  
          
        # Add the patch to the Axes  
        ax.add_patch(bbox)  
          
        # Get the class name  
        class_name = class_mapping[row['class_id']]  
          
        # Annotate with class name  
        plt.text(row['xmin'], row['ymin'] - 10, f'Class: {class_name}',  
                 color='red', fontsize=12, weight='bold')  
          
        # Display the plot  
        plt.show()  
  
plot_images_with_bboxes(df)  


In [ ]:
from sklearn.model_selection import train_test_split  
  
# Split the DataFrame into training (80%), validation (10%), and test (10%) sets  
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)  
  
print(f"Training set size: {len(train_df)}")  
print(f"Validation set size: {len(val_df)}")  
print(f"Test set size: {len(test_df)}")  
  
# Function to generate data for the model  
def data_generator(df, batch_size, img_size=(224, 224)):  
    while True:  
        for start in range(0, len(df), batch_size):  
            end = min(start + batch_size, len(df))  
            batch_df = df[start:end]  
            images = []  
            bbox_targets = []  
            class_targets = []  
            for idx, row in batch_df.iterrows():  
                img = Image.open(row['path_to_frame'])  
                img = img.resize(img_size)  
                img = np.array(img) / 255.0  
                images.append(img)  
                bbox_targets.append([row['xmin'], row['ymin'], row['xmax'], row['ymax']])  
                class_targets.append(row['class_id'])  
            yield np.array(images), {'bbox_output': np.array(bbox_targets), 'class_output': np.array(class_targets)}  


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, Flatten, Dense, Dropout  
from tensorflow.keras.models import Model  
  
# Define the CNN model with separate outputs for bbox and class ID  
def create_model():  
    inputs = Input(shape=(224, 224, 3))  
      
    x = Conv2D(32, (3, 3), activation='relu')(inputs)  
    x = MaxPooling2D((2, 2))(x)  
    x = Conv2D(64, (3, 3), activation='relu')(x)  
    x = MaxPooling2D((2, 2))(x)  
    x = Conv2D(128, (3, 3), activation='relu')(x)  
    x = MaxPooling2D((2, 2))(x)  
    x = Conv2D(128, (3, 3), activation='relu')(x)  
    x = MaxPooling2D((2, 2))(x)  
    x = Flatten()(x)  
    x = Dense(128, activation='relu')(x)  
    x = Dropout(0.5)(x)  
      
    bbox_output = Dense(4, activation='linear', name='bbox_output')(x)  
    class_output = Dense(3, activation='softmax', name='class_output')(x)  
      
    model = Model(inputs=inputs, outputs=[bbox_output, class_output])  
      
    model.compile(optimizer='adam',  
                  loss={'bbox_output': 'mean_squared_error', 'class_output': 'sparse_categorical_crossentropy'},  
                  metrics={'bbox_output': 'accuracy', 'class_output': 'accuracy'})  
      
    return model  
  
model = create_model()  
model.summary()  

In [ ]:
#batch_size = 32  
batch_size = 3  
  
train_gen = data_generator(train_df, batch_size)  
val_gen = data_generator(val_df, batch_size)  
  
steps_per_epoch = len(train_df) // batch_size  
validation_steps = len(val_df) // batch_size  
  
model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=val_gen, validation_steps=validation_steps)  


In [ ]:
test_gen = data_generator(test_df, batch_size)  
  
# Evaluate on the test set  
test_steps = len(test_df) // batch_size  
results = model.evaluate(test_gen, steps=test_steps)  
print(f"Test Loss: {results[0]}")  
print(f"Test Bounding Box Accuracy: {results[1]}")  
print(f"Test Class Accuracy: {results[2]}")  


In [ ]:
import matplotlib.pyplot as plt  
import matplotlib.patches as patches  
  
# Define class names mapping for display  
class_names = {0: 'apple', 1: 'banane', 2: 'orange'}  
  
# Function to calculate IoU  
def calculate_iou(box1, box2):  
    x1, y1, x2, y2 = box1  
    x1_p, y1_p, x2_p, y2_p = box2  
  
    xi1 = max(x1, x1_p)  
    yi1 = max(y1, y1_p)  
    xi2 = min(x2, x2_p)  
    yi2 = min(y2, y2_p)  
  
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)  
  
    box1_area = (x2 - x1) * (y2 - y1)  
    box2_area = (x2_p - x1_p) * (y2_p - y1_p)  
  
    union_area = box1_area + box2_area - inter_area  
  
    iou = inter_area / union_area  
    return iou  
  
# Function to plot bounding boxes on images with IoU  
def plot_bounding_boxes_with_iou(image, true_bbox, true_class, pred_bbox, pred_class):  
    fig, ax = plt.subplots(1)  
    ax.imshow(image)  
  
    # Draw ground truth bounding box  
    rect = patches.Rectangle((true_bbox[0], true_bbox[1]), true_bbox[2] - true_bbox[0], true_bbox[3] - true_bbox[1],  
                             linewidth=2, edgecolor='g', facecolor='none')  
    ax.add_patch(rect)  
    plt.text(true_bbox[0], true_bbox[1] - 10, f'True: {class_names[true_class]}', color='green', fontsize=12, weight='bold')  
  
    # Draw predicted bounding box  
    rect = patches.Rectangle((pred_bbox[0], pred_bbox[1]), pred_bbox[2] - pred_bbox[0], pred_bbox[3] - pred_bbox[1],  
                             linewidth=2, edgecolor='r', facecolor='none')  
    ax.add_patch(rect)  
    plt.text(pred_bbox[0], pred_bbox[1] - 30, f'Pred: {class_names[pred_class]}', color='red', fontsize=12, weight='bold')  
  
    # Calculate IoU  
    iou = calculate_iou(true_bbox, pred_bbox)  
    plt.text(10, 10, f'IoU: {iou:.2f}', color='white', fontsize=12, weight='bold', bbox=dict(facecolor='blue', alpha=0.8))  
  
    plt.show()  
  
# Loop through the test set and plot images with bounding boxes  
for idx, row in test_df.iterrows():  
    img_path = row['path_to_frame']  
    img = Image.open(img_path).resize((224, 224))  
    img_array = np.array(img) / 255.0  
    img_array = np.expand_dims(img_array, axis=0)  
  
    # Predict bounding box and class  
    pred_bbox, pred_class = model.predict(img_array)  
    pred_bbox = pred_bbox[0]  
    pred_class = np.argmax(pred_class[0])  
  
    # Get true bounding box and class  
    true_bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]  
    true_class = row['class_id']  
  
    # Plot the image with bounding boxes and IoU  
    plot_bounding_boxes_with_iou(np.array(img), true_bbox, true_class, pred_bbox, pred_class)  
